In [34]:
#%pip install -r requirements.txt
from IPython.display import clear_output
clear_output()
from enum import Enum, auto
from dataclasses import dataclass, fields
import typing
from typing import Any, Tuple
import random
from pprint import pprint
%load_ext mypy_ipython

The mypy_ipython extension is already loaded. To reload it, use:
  %reload_ext mypy_ipython


In [2]:
def weighted_if(weight, out1, out2):
    return out1 if random.random() < weight else out2

In [4]:
@dataclass(frozen=True, eq=True)
class Allele:
    value: Any
    dominant: bool

In [35]:
class BeeGender(Enum):
    DRONE = auto()
    PRINCESS = auto()
    QUEEN = auto()

class BeeSpecies(Enum):
    FOREST      = auto()
    MEADOWS     = auto()
    COMMON      = auto()
    CULTIVATED  = auto()
    MAJESTIC    = auto()
    NOBLE       = auto()
    IMPERIAL    = auto()
    DILIGENT    = auto()
    UNWEARY     = auto()
    INDUSTRIOUS = auto()
    # COMMON    = auto()
    # COMMON    = auto()
    # COMMON    = auto()
    # COMMON    = auto()
    
bs = BeeSpecies
mutations = {
    (Allele(bs.FOREST,     True), Allele(bs.MEADOWS, True)): (Allele(bs.COMMON,     True), 1),
    (Allele(bs.FOREST,     True), Allele(bs.COMMON,  True)): (Allele(bs.CULTIVATED, True), 1),
    (Allele(bs.COMMON,     True), Allele(bs.MEADOWS, True)): (Allele(bs.CULTIVATED, True), 1),
    (Allele(bs.CULTIVATED, True), Allele(bs.COMMON,  True)): (Allele(bs.MAJESTIC,   True), 1),
}
mutations.update({ (key[1], key[0]) : mutations[key] for key in mutations})
pprint(mutations)
del bs

{(Allele(value=<BeeSpecies.FOREST: 1>, dominant=True), Allele(value=<BeeSpecies.MEADOWS: 2>, dominant=True)): (Allele(value=<BeeSpecies.COMMON: 3>, dominant=True),
                                                                                                               1),
 (Allele(value=<BeeSpecies.MEADOWS: 2>, dominant=True), Allele(value=<BeeSpecies.COMMON: 3>, dominant=True)): (Allele(value=<BeeSpecies.CULTIVATED: 4>, dominant=True),
                                                                                                               1),
 (Allele(value=<BeeSpecies.COMMON: 3>, dominant=True), Allele(value=<BeeSpecies.FOREST: 1>, dominant=True)): (Allele(value=<BeeSpecies.CULTIVATED: 4>, dominant=True),
                                                                                                              1),
 (Allele(value=<BeeSpecies.FOREST: 1>, dominant=True), Allele(value=<BeeSpecies.COMMON: 3>, dominant=True)): (Allele(value=<BeeSpecies.CULTIVATED: 4>, domina

In [18]:
Gene = Tuple[Allele, Allele]

@dataclass
class Genes:
    species: Gene

In [24]:
class Bee:
    def __init__(self, gender: BeeGender, genes: Genes):
        self.gender = gender
        self.genes = genes
    
    @staticmethod
    def mutate(allele1, allele2):
        mut = mutations.get((allele1, allele2))
        if mut is not None and random.random() < mut[1]: # mutation possible and with probability [1]
            return mut[0] # gene from mutation
        return weighted_if(0.5, allele1, allele2) # random gene if not mutated
    
    @staticmethod
    def crossingover(genes1, genes2):
        l = []
        genes1_dict = vars(genes1)
        genes2_dict = vars(genes2)
        for key1, key2 in zip(genes1_dict, genes2_dict):
            g1 = genes1_dict[key1]
            g2 = genes2_dict[key2]
            
            allele11 = weighted_if(0.5, *g1)
            allele12 = weighted_if(0.5, *g2)
            allele21 = weighted_if(0.5, *g1)
            allele22 = weighted_if(0.5, *g2)
            
            allele1 = Bee.mutate(allele11, allele12)
            allele2 = Bee.mutate(allele21, allele22)
            
            l.append((allele1, allele2))
        return Genes(*l)

class Queen(Bee):
    def __init__(self, genes):
        super().__init__(BeeGender.QUEEN, genes)
    
    def die(self):
        return [
            Princess(self.genes),
            Drone(self.genes),
            Drone(self.genes)
        ]
    
class Princess(Bee):
    def __init__(self, genes):
        super().__init__(BeeGender.PRINCESS, genes)
    
    def mate(self, other: 'Drone') -> Queen:
        g = self.crossingover(self.genes, other.genes)
        return Queen(g)

class Drone(Bee):
    def __init__(self, genes):
        super().__init__(BeeGender.DRONE, genes)

In [29]:
pf = Princess(Genes((Allele(BeeSpecies.FOREST, True), Allele(BeeSpecies.FOREST, True))))
dm = Drone(Genes((Allele(BeeSpecies.MEADOWS, True), Allele(BeeSpecies.MEADOWS, True))))
qc = pf.mate(dm)
pc, dc, dc2 = qc.die()
qcu = pc.mate(dm)
pcu, dcu, dcu2 = qcu.die()
pcu.mate(pc).genes

Genes(species=(Allele(value=<BeeSpecies.COMMON: 3>, dominant=True), Allele(value=<BeeSpecies.COMMON: 3>, dominant=True)))


Genes(species=(Allele(value=<BeeSpecies.MAJESTIC: 5>, dominant=True), Allele(value=<BeeSpecies.MAJESTIC: 5>, dominant=True)))

In [30]:
%mypy

    print(qc.genes)
error: Cannot determine type of "genes"
    print(qc.genes)
error: Cannot determine type of "genes"
    print(qc.genes)
error: Cannot determine type of "genes"
    print(qc.genes)
error: Cannot determine type of "genes"
    print(qc.genes)
error: Cannot determine type of "genes"
    print(qc.genes)
error: Cannot determine type of "genes"
Found 6 errors in 1 file (checked 1 source file)


Type checking failed
